In [ ]:
### Author: Julian Viereck
### Date: 13 Jan 2020
### Put together full step adjustment algorithm for stepping in place.

import numpy as np
np.set_printoptions(precision=2, suppress=True)

import time

import os
import rospkg
import pybullet as p
import pinocchio as se3
from pinocchio.utils import se3ToXYZQUAT

from robot_properties_solo.config import Solo12Config
from robot_properties_solo.quadruped12wrapper import Quadruped12Robot

from py_blmc_controllers.solo_centroidal_controller import SoloCentroidalController
from py_blmc_controllers.solo_impedance_controller import SoloImpedanceController 

import dynamic_graph as dg
from dg_tools.utils import *
from dynamic_graph_manager.dg_reactive_planners import StepperHead, DcmVrpPlanner
from dynamic_graph.sot.core.switch import SwitchVector
import dynamic_graph.sot.dynamics_pinocchio as dg_pinocchio

from pinocchio.utils import zero
from matplotlib import pyplot as plt

from py_dcm_vrp_planner.planner import DcmVrpPlanner as PyDcmVrpPlanner



#######################################################################################

# Create a robot instance. This initializes the simulator as well.
robot = Quadruped12Robot()
tau = np.zeros(12)

p.resetDebugVisualizerCamera(1.2, 135, -30, (0,0,0.1))

Solo12Config.initial_configuration[0] = 0.0
Solo12Config.initial_configuration[2] = 0.25

In [ ]:
import six
from dynamic_graph.signal_base import objectToString, SignalBase
class BaseOperatorSignal(SignalBase):
    def __init__(self, sig):
        super(BaseOperatorSignal, self).__init__(obj=sig.obj)

    def _op(self, other, op_name, op_maps):
        # TODO: If given other=signal, try to implicity convert to OperatorSignal
        #   by inferring the type from the signal name.
        #   Eg: other.name = 'SinEntity()::output(double)::sout'.
        
        for k, v in six.iteritems(op_maps):
            if isinstance(other, k):
                op = v['op']('')
                if 'value' in v:
                    op.signal(v['sinB']).value = other
                else:
                    dg.plug(other, op.signal(v['sinB']))
                dg.plug(self, op.signal(v['sinA']))
                return v['return_wrap'](self, op, other)
                    
        raise ValueError('Unsupported "%s" operation with "%s"' % (op_name, str(other)))
        
    def __add__(self, other):
        return self._op(other, '__add__', self._add)
        
    def __mul__(self, other):
        return self._op(other, '__mul__', self._mul)
        
    def __sub__(self, other):
        return self._op(other, '__sub__', self._sub)
        
    def __div__(self, other):
        return self.__truediv__(other)
        
    def __truediv__(self, other):
        return self._op(other, '__truediv__', self._truediv)
            
    def __repr__(self):
        return type(self).__name__ + ': value=' + str(self.value)
        
def opdef(op, return_wrap, use_value=False, use_sin1=True):
    """Creates a definition for how to use the operator.
    
    Args:
      op: Operator class to create.
      return_wrap: Function used to create the returned output.
      use_value: (optional) If provided, put the `other` as value
      use_sin1: If True, uses `sin1` and `sin2`, if False use `sin0` 
          and `sin1` for the operator.
      """
    d = {
        'op': op,
        'return_wrap': return_wrap
    }
    if use_value:
        d['value'] = True
    if use_sin1:
        d['sinA'] = 'sin1'
        d['sinB'] = 'sin2'
    else:
        d['sinA'] = 'sin0'
        d['sinB'] = 'sin1'
    return d

class VectorSignal(BaseOperatorSignal):
    def __init__(self, sig, length=None):
        if isinstance(sig, list) or isinstance(sig, tuple) or isinstance(sig, np.ndarray):
            if length is None:
                length = len(sig)
            if len(sig) != length:
                raise ValueError('VectorSignal: Length wrong for provided vector.')
            sig = constVector(sig)
            
        if length is None:
            raise ValueError('VectorSignal: Need to provide length of vector signal.')

        super(VectorSignal, self).__init__(sig)
        self.length = length
        
        self._add = {
            VectorSignal: opdef(Add_of_vector, lambda s, op, other: VectorSignal(op.sout, s.length))
        }
        
        self._sub = {
            VectorSignal: opdef(Substract_of_vector, lambda s, op, other: VectorSignal(op.sout, s.length))
        }
        
    def __getitem__(self, val):
        if isinstance(val, slice):
            if val.step != 1 and val.step != None:
                raise ValueError('VectorSignal.__getitem__: Only support slice with step 1.')
            if val.start < 0 or val.start > self.length:
                raise ValueError('VectorSignal.__getitem__: Slice start index "%s" out of range' % (str(val.start)))
            if val.stop < 0 or val.stop > self.length:
                raise ValueError('VectorSignal.__getitem__: Slice stop index "%s" out of range.' % (str(val.start)))
            
            op = Selec_of_vector('')
            op.selec(val.start, val.stop)
            dg.plug(self, op.sin)
            return VectorSignal(op.sout, val.stop - val.start)
        elif isinstance(val, int):
            op = Component_of_vector('')
            op.setIndex(val)
            plug(self, op.sin)
            return DoubleSignal(op.sout)

        raise ValueError('VectorSignal.__getitem__: Unsupported item type "%s"' % (str(val)))
            
    def concat(self, other):
        if not isinstance(other, VectorSignal):
            raise ValueError('VectorSignal.concat(other): Expecting `other` to be of type VectorSignal but got "%s".' % str(other))
        
        op = Stack_of_vector("")
        op.selec1(0, self.length)
        op.selec2(0, other.length)
        plug(self, op.sin1)
        plug(other, op.sin2)
        return VectorSignal(op.sout, self.length + other.length)
    
            
class DoubleSignal(BaseOperatorSignal):
    def __init__(self, sig):
        if isinstance(sig, int) or isinstance(sig, float):
            sig = constDouble(sig)
        super(DoubleSignal, self).__init__(sig)

        return_wrap = lambda s, op, other: DoubleSignal(op.sout)
        
        self._add = {
            float: opdef(Add_of_double, return_wrap, use_value=True),
            DoubleSignal: opdef(Add_of_double, return_wrap)
        }
        
        self._sub = {
            float: opdef(Substract_of_double, return_wrap, use_value=True),
            DoubleSignal: opdef(Substract_of_double, return_wrap)
        }
        
        self._mul = {
            float: opdef(Multiply_of_double, return_wrap, use_value=True, use_sin1=False),
            DoubleSignal: opdef(Multiply_of_double, return_wrap, use_sin1=False),
            VectorSignal: opdef(Multiply_double_vector, lambda s, op, other: VectorSignal(op.sout, other.length))
        }
        
        self._truediv = {
            float: opdef(Division_of_double, return_wrap, use_value=True),
            DoubleSignal: opdef(Division_of_double, return_wrap)
        }
        
    def vector(self):
        return self * VectorSignal([1.])      
        
        
# Minimal unit tests.
c = DoubleSignal(1)
v = VectorSignal([1, 2.])
assert((c * v).length == 2)

t = (c / c)
t.recompute(1)
assert(t.value == 1.)

v = VectorSignal([1, 2, 3, 4], 4)
k = v[1:3]
k.recompute(1)
assert(k.value == (2., 3.))

t = v[1]
t.recompute(1)
assert(t.value == 2)

t = v.concat(VectorSignal([5, 6]))
t.recompute(1)
assert(t.value == (1, 2, 3, 4, 5, 6))

d = c.vector()
d.recompute(1)
assert(d.value == (1.,))

In [ ]:
arr = lambda a: np.array(a).reshape(-1)
mat = lambda a: np.matrix(a).reshape((-1, 1))
total_mass = sum([i.mass for i in robot.pin_robot.model.inertias[1:]])

def zero_cnt_gain(kp, cnt_array):
    gain = np.array(kp).copy()
    for i, v in enumerate(cnt_array):
        if v == 1:
            gain[3*i:3*(i + 1)] = 0.

    return gain

#######################################################

warmup = 250
x_des = np.array(4*[0.0, 0.0, 0.])
xd_des = 4*[0,0,0] 
kp = np.array(12 * [50.])
kp[2::3] = 100
kd = 12 * [5.]
u = [0, 0]
##################################################################################

ht = 0.20
x_com_cent = [0.0, 0.0, ht]
xd_com_cent = [0.0, 0.0, 0.0]

x_ori = [0., 0., 0., 1.]
x_angvel = [0., 0., 0.]
cnt_array = [1, 1, 1, 1]

solo_leg_ctrl = SoloImpedanceController(robot)
centr_controller = SoloCentroidalController(robot.pin_robot, total_mass,
        mu=0.6, kc=100, dc=10, kb=[5000, 5000, 500], db=5.,
        eff_ids=robot.pinocchio_endeff_ids)

# Positions of the foot at zero positions of solo12.
x0 = np.array([
    0.195, 0.147, 0,
    0.195, -0.147, 0,
    -0.195, 0.147, 0,
    -0.195, -0.147, 0,
])

x_des = x0.copy()

#################################################################################

l_min = -0.15
l_max = 0.15
w_min = -0.15
w_max = 0.15
t_min = 0.1
t_max = 0.3
v_des = [0.5, 0.0]
l_p = 0

dcm_vrp_planner = PyDcmVrpPlanner(l_min, l_max, w_min, w_max, t_min, t_max, v_des, l_p, ht)

W = [100, 100, 1, 1000, 1000, 100] # weight on [step length_x , step_length_y, step time, dcm_offeset_x, dcm_offeset_y]

#######################################################################################

sh = StepperHead('stepper_head')
dcm_planner = DcmVrpPlanner('dcm_planner')
dg_com = constVector([0, 0, 0])
dg_dcom = constVector([0, 0, 0])
dg_des_vel = constVector([0.5, 0., 0.])
dg_world_M_base = constVector(7 * [0.])

dg.plug(dg_com, dcm_planner.com_sin)
dg.plug(dg_dcom, dcm_planner.com_vel_sin)
dg.plug(dg_des_vel, dcm_planner.desired_velocity_sin)
dg.plug(dg_world_M_base, dcm_planner.world_M_base_sin)

dcm_planner.initializeParamVector([
    l_min, l_max,
    w_min, w_max,
    t_min, t_max,
    l_p, ht,
    # u_x, u_y, tau, b_x, b_y, psi_0, psi_1, psi_2, psi_3
    100, 100, 1, 1000, 1000, 1e8, 1e8, 1e8, 1e8
])

# Connecting the dcm_planner to stepper_head.
dg.plug(dcm_planner.duration_before_step_landing_sout, sh.duration_before_step_landing_sin)
dg.plug(dcm_planner.next_step_location_sout, sh.next_step_location_sin)

# Connecting stepper_head to dcm_planner.
dg.plug(sh.current_step_location_sout, dcm_planner.current_step_location_sin)
dg.plug(sh.time_from_last_step_touchdown_sout, dcm_planner.time_from_last_step_touchdown_sin)
dg.plug(sh.is_left_leg_in_contact_sout, dcm_planner.is_left_leg_in_contact_sin)


# Simple trajectory generation.
dg_z_max = DoubleSignal(0.1)
dg_u_old = VectorSignal(sh.old_step_location_sout, 3)
dg_u = VectorSignal(dcm_planner.next_step_location_sout, 3)

dg_t = DoubleSignal(sh.time_from_last_step_touchdown_sout)
dg_t_end = DoubleSignal(dcm_planner.duration_before_step_landing_sout)

dg_t_quot = dg_t / dg_t_end
dg_x_foot_des_air_xy = (dg_u_old + dg_t_quot * (dg_u - dg_u_old))[0:2]
dg_x_foot_des_air_z = dg_z_max * DoubleSignal(sin_doub(dg_t_quot * np.pi))

dg_x_foot_des_air = dg_x_foot_des_air_xy.concat(dg_x_foot_des_air_z.vector())
dg_x_foot_des_ground = VectorSignal(dcm_planner.next_step_location_sout, 3)

# Concatinate desired position for all endeffector.
# Provide configuration for left or right state.
dg_xdes_left = dg_x_foot_des_air.concat(dg_x_foot_des_ground).concat(dg_x_foot_des_ground).concat(dg_x_foot_des_air)
dg_xdes_right = dg_x_foot_des_ground.concat(dg_x_foot_des_air).concat(dg_x_foot_des_air).concat(dg_x_foot_des_ground)

# Switch the desired leg position and contact configurations.
switch_xdes = SwitchVector("control_switch_xdes")
switch_xdes.setSignalNumber(2) # we want to switch between 2 signals
plug(dg_xdes_left, switch_xdes.sin0)
plug(dg_xdes_right, switch_xdes.sin1)
plug(sh.is_left_leg_in_contact_sout, switch_xdes.selection)

switch_cnt_array = SwitchVector("control_switch_cnt_array")
switch_cnt_array.setSignalNumber(2) # we want to switch between 2 signals
plug(VectorSignal([0, 1, 1, 0]), switch_cnt_array.sin0)
plug(VectorSignal([1, 0, 0, 1]), switch_cnt_array.sin1)
plug(sh.is_left_leg_in_contact_sout, switch_cnt_array.selection)

# Offset the leg positions with the initial positions of the legs.
dg_x_des = VectorSignal(switch_xdes.sout, 12) + VectorSignal(x0, 12)
        
# – 	__sub__(self, other)
# * 	__mul__(self, other)
# / 	__truediv__(self, other)

In [ ]:
robot_dg = dg_pinocchio('dg_solo12)
                        

In [ ]:
# Reset the robot to some initial state.
q0 = np.matrix(Solo12Config.initial_configuration).T
dq0 = np.matrix(Solo12Config.initial_velocity).T
robot.reset_state(q0, dq0)

solo_leg_ctrl = SoloImpedanceController(robot)

# Run the simulator for 100 steps
t = 0
t_gap = 0.05
t_end = t_max
n = 1
u_current_step = [0.0, 0.0]
u_old = [0.0, 0.0]
x_com = np.array([0.0, 0.0]) 
xd_com = np.array([0.0, 0.0]) 

class Local2WorldTransform(object):
    """
    Helper class for doing SE3 transformation. Transforms from the world frame
    into the base frame while using only the z/yaw rotation.
    """
    def __init__(self, q):
        robot_se3 = se3.XYZQUATToSe3(q[:7])
        rotation = se3.utils.rpyToMatrix(np.matrix([
            0, 0, float(se3.utils.matrixToRpy(robot_se3.rotation)[2])
        ]).T)

        self.trans_ = se3.SE3(rotation, robot_se3.translation)
        self.rotat_ = se3.SE3(rotation, np.matrix([0, 0, 0]).T)
        
    def transInv(self, vec):
        vec = self.trans_.actInv(np.matrix([vec[0], vec[1], 0]).T)
        return np.array(vec[:2]).reshape(-1)
    
    def rotInv(self, vec):
        vec = self.rotat_.actInv(np.matrix([vec[0], vec[1], 0]).T)
        return np.array(vec[:2]).reshape(-1)
    
    def trans(self, vec):
        vec = self.trans_.act(np.matrix([vec[0], vec[1], 0]).T)
        return np.array(vec[:2]).reshape(-1)
    
    def rot(self, vec):
        vec = self.rotat_.act(np.matrix([vec[0], vec[1], 0]).T)
        return np.array(vec[:2]).reshape(-1)

def sig_get(sig, t):
    sig.recompute(t)
    return sig.value

## for plotting
plt_opt = []
plt_foot = []
plt_com = []
plt_force = []
plt_tau = []
plt_cnt = []
plt_xdes = []
for i in range(2000):
    q, dq = robot.get_state()
    robot.pin_robot.com(q, dq)
    x_com = np.array(robot.pin_robot.data.com[0][0:3]).reshape(-1)
    xd_com = np.array(robot.pin_robot.data.vcom[0][0:3]).reshape(-1)

#     if i > 875 and i < 925:
#         force = np.array([0,10,0])
#         p.applyExternalForce(objectUniqueId=robot.robotId, linkIndex=-1, forceObj=force, \
#                         posObj=[0.2 + q[0],0.0,0], flags = p.WORLD_FRAME)

#     if i > 875 and i < 1000:
#         time.sleep(0.05)        

    # Very simple stepping:
    if i >= warmup:
        floor_height = -0.00
        tend_scale = 0.85
        
        # TODO: Plug these values to dg-pinocchio.
        dg_com.value = x_com
        dg_dcom.value = xd_com
        dg_world_M_base.value = np.array(q[:7]).reshape(-1)
        
        # Read the desired position and step locations.
        x_des = np.array(sig_get(dg_x_des, i))
        cnt_array = sig_get(switch_cnt_array.sout, i)

    # Convert x_des from world to impedance-local frame
    x_des_local = x_des.copy()
    plt_xdes.append(x_des.copy())
    
    for j in range(4):
        imp = solo_leg_ctrl.imps[j]
        x_des_local[3*j:3*(j+1)] -= np.array(imp.pin_robot.data.oMf[imp.frame_end_idx].translation).reshape(-1)
        x_des_local[3*j:3*(j+1)] += np.array(imp.compute_distance_between_frames(q)).reshape(-1)
        
    x_com_cent[0] = 0.001 * (max(i - warmup, 0)) * v_des[0]
    w_com = centr_controller.compute_com_wrench(t, q, dq, x_com_cent, xd_com_cent, x_ori, x_angvel)
    w_com[2] += total_mass * 9.81
    F = centr_controller.compute_force_qp(i, q, dq, cnt_array, w_com)
    tau = solo_leg_ctrl.return_joint_torques(q,dq,
                                             zero_cnt_gain(kp, cnt_array),
                                             zero_cnt_gain(kd, cnt_array), x_des_local, xd_des, F)
    plt_tau.append(tau)
    plt_force.append(F)
    plt_cnt.append(cnt_array)
    robot.send_joint_command(tau)
    p.stepSimulation()
    
plt.plot(np.array(plt_tau).reshape(-1, 12))